# Empirical distribution functions for blocks and transactions

## Set up

In [1]:
suppressMessages({
    require(data.table, quietly=TRUE)
    require(geosphere, quietly=TRUE)
    require(ggplot2, quietly=TRUE)
    require(ggraph, quietly=TRUE)
    require(igraph, quietly=TRUE)
    require(magrittr, quietly=TRUE)
    require(maps, quietly=TRUE)
    require(RColorBrewer, quietly=TRUE)
    require(tidygraph, quietly=TRUE)
})

## Blocks

### Read data

In [2]:
ledgerOps <- fread("../apply-reapply/ledger-ops-65836843.csv.gz", stringsAsFactors=TRUE)
ledgerOps %>% summary

    slot_no             block_no           epoch_no     epoch_slot_no   
 Min.   : 65836984   Min.   : 7477994   Min.   :350.0   Min.   :     0  
 1st Qu.: 89833176   1st Qu.: 8642234   1st Qu.:405.0   1st Qu.:108302  
 Median :113740091   Median : 9806475   Median :460.0   Median :216127  
 Mean   :113666120   Mean   : 9806475   Mean   :460.2   Mean   :216087  
 3rd Qu.:137528206   3rd Qu.:10970716   3rd Qu.:515.0   3rd Qu.:323960  
 Max.   :161151612   Max.   :12134956   Max.   :570.0   Max.   :431999  
                                                                        
                                                            block_hash     
 0000010581c9e6dd574a1c0c2d087e54d6012d773eac3cd0f67e4b40109e28ae:      1  
 000003b6bbc9fbc1cde9bd40dd9c2f0a4aae216e7b0de4da85fa29cae90c670f:      1  
 000009a39da244fe4fa71fb6d803c56b813373f7ca2a11bfc54c6ea950cb7c24:      1  
 00000ac813d4152a792610999e890ba56c58cec5a5c299754fc0b12ce8ca33d6:      1  
 00000dc704b3c1c7f2a93d8d33479208281

### Compute quantiles

In [3]:
txsQuantiles <- ledgerOps[, quantile(`txs`, probs=seq(0, 1, length.out=10))]
txsQuantiles

0% 11.11111% 22.22222% 33.33333% 44.44444% 55.55556% 66.66667% 77.77778% 
        0         1         3         6         8        12        16        22 
88.88889%      100% 
       32       385

In [4]:
sizeQuantiles <- ledgerOps[, quantile(`txs_size`, probs=seq(0, 1, length.out=10))]
sizeQuantiles

0% 11.11111% 22.22222% 33.33333% 44.44444% 55.55556% 66.66667% 77.77778% 
        0      1165      4285      9449     16011     24533     36536     54513 
88.88889%      100% 
    81655     89053

In [5]:
applyQuantiles <- ledgerOps[, quantile(`apply_mut`, probs=seq(0, 1, length.out=10))]
applyQuantiles

0% 11.11111% 22.22222% 33.33333% 44.44444% 55.55556% 66.66667% 77.77778% 
      255       730      1709      2894      4330      6084      8332     11238 
88.88889%      100% 
    14632  17649757

In [6]:
reapplyQuantiles <- ledgerOps[, quantile(`reapply_mut`, probs=seq(0, 1, length.out=10))]
reapplyQuantiles

0% 11.11111% 22.22222% 33.33333% 44.44444% 55.55556% 66.66667% 77.77778% 
       46       212       325       443       582       755       980      1279 
88.88889%      100% 
     1722  17413702

### Summarize by quantiles

In [7]:
blockEdf <- ledgerOps[
    ,
    .(
        `Tx count`=mean(`txs`),
        `Block size [kB]`=mean(`txs_size`)/1e3,
        `Apply [ms]`=mean(`apply_mut`)/1e3,
        `Reapply [ms]`=mean(`reapply_mut`)/1e3,
        `Number of blocks`=.N
    ),
    .(
        `Tx count quantile`=`txs`,
        `Block size quantile`=cut(`txs_size`, breaks=sizeQuantiles, labels=FALSE, include.lowest=TRUE),
        `Apply quantile`=cut(`apply_mut`, breaks=applyQuantiles, labels=FALSE, include.lowest=TRUE),
        `Reapply quantile`=cut(`reapply_mut`, breaks=reapplyQuantiles, labels=FALSE, include.lowest=TRUE)
    )
][, .(
    `Tx count`,
    `Block size [kB]`,
    `Apply [ms]`,
    `Reapply [ms]`,
    `Fraction of blocks [%/100]`=`Number of blocks`/sum(`Number of blocks`)
)][order(`Tx count`, `Block size [kB]`, `Apply [ms]`, `Reapply [ms]`)]
blockEdf %>% summary

    Tx count      Block size [kB]   Apply [ms]         Reapply [ms]      
 Min.   :  0.00   Min.   : 0.00   Min.   :   0.3436   Min.   :   0.0550  
 1st Qu.:  7.00   1st Qu.:13.72   1st Qu.:   4.1614   1st Qu.:   0.6798  
 Median : 15.00   Median :30.23   Median :   7.4040   Median :   1.1194  
 Mean   : 26.64   Mean   :37.96   Mean   :  10.5994   Mean   :   2.7112  
 3rd Qu.: 29.00   3rd Qu.:63.69   3rd Qu.:  12.5030   3rd Qu.:   1.8727  
 Max.   :385.00   Max.   :89.02   Max.   :1328.2510   Max.   :1428.9370  
 Fraction of blocks [%/100]
 Min.   :2.100e-07         
 1st Qu.:8.600e-07         
 Median :4.940e-06         
 Mean   :1.066e-04         
 3rd Qu.:4.638e-05         
 Max.   :3.384e-02         

### Check that means are preserved

In [8]:
blockEdf[
    ,
    .(
        `Tx count`=sum(`Tx count`*`Fraction of blocks [%/100]`),
        `Block size [kB]`=sum(`Block size [kB]`*`Fraction of blocks [%/100]`),
        `Apply [ms]`=sum(`Apply [ms]`*`Fraction of blocks [%/100]`),
        `Reapply [ms]`=sum(`Reapply [ms]`*`Fraction of blocks [%/100]`)
    )
]

Tx count,Block size [kB],Apply [ms],Reapply [ms]
<dbl>,<dbl>,<dbl>,<dbl>
14.19234,30.23077,7.223127,1.006942


In [9]:
ledgerOps[
    ,
    .(
        `Tx count`=mean(`txs`),
        `Block size [kB]`=mean(`txs_size`)/1e3,
        `Apply [ms]`=mean(`apply_mut`)/1e3,
        `Reapply [ms]`=mean(`reapply_mut`)/1e3
    )
]

Tx count,Block size [kB],Apply [ms],Reapply [ms]
<dbl>,<dbl>,<dbl>,<dbl>
14.19234,30.23077,7.223127,1.006942


### Write TSV file

In [10]:
fwrite(blockEdf, "block-edf.tsv", sep="\t")